In [1]:
import numpy as np
import keras
import string
with open('/Users/AsyaRodigina/hse/SQuAD/SQuAD/lab-5/train.txt') as f:
    content = f.readlines()
vec = list()
for line in content:
    one_line = line.split()
    vec.append(one_line)
almost_data = list()
for elem in vec:
    q = list()
    q.append(list(elem[0]))
    q.append(elem[1].split('_'))
    if (len(elem) == 2):
        almost_data.append(q)
    else:
        almost_data.append(q)
        q = list()
        q.append(list(elem[0]))
        q.append(elem[2].split('_'))
        almost_data.append(q)

letters = {}
phs = {}
token = 1
counter = 1
for elem in almost_data:
    for let in elem[0]:
        if let not in letters:
            letters[let] = token
            token += 1
    for ph in elem[1]:
        if ph not in phs:
            phs[ph] = counter
            counter +=1

Using TensorFlow backend.


In [4]:
almost_data[1]

[['M', 'I', 'N', 'D', 'I', 'N', 'G'], ['M', 'AY', 'N', 'D', 'IH', 'NG']]

In [3]:
to_phs = {}
counter = 1
for elem in phs:
    to_phs[counter] = elem
    counter +=1
to_phs[40] = ''
to_phs

{1: 'L',
 2: 'AH',
 3: 'M',
 4: 'Y',
 5: 'UW',
 6: 'AY',
 7: 'N',
 8: 'D',
 9: 'IH',
 10: 'NG',
 11: 'S',
 12: 'T',
 13: 'R',
 14: 'P',
 15: 'K',
 16: 'EH',
 17: 'AA',
 18: 'F',
 19: 'ER',
 20: 'EY',
 21: 'AE',
 22: 'Z',
 23: 'G',
 24: 'B',
 25: 'SH',
 26: 'V',
 27: 'OW',
 28: 'AO',
 29: 'IY',
 30: 'W',
 31: 'HH',
 32: 'JH',
 33: 'CH',
 34: 'TH',
 35: 'AW',
 36: 'OY',
 37: 'UH',
 38: 'ZH',
 39: 'DH',
 40: ''}

In [3]:
x_data = np.zeros((88628, 35, len(letters) + 2))
y_data = np.zeros((88628, 35, len(to_phs) + 1))
len(letters) + 2

30

In [4]:
import copy
for i in range(len(almost_data)):
    for j in range(len(almost_data[i][0])):
        x_data[i][j][letters[almost_data[i][0][j]]] = 1
    for j in range(len(almost_data[i][0]), 35):
        x_data[i][j][29] = 1
    
    for j in range(len(almost_data[i][1])):
        y_data[i][j][phs[almost_data[i][1][j]]] = 1
    for j in range(len(almost_data[i][1]), 35):
        y_data[i][j][40] = 1

In [25]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, BatchNormalization
from keras.layers.wrappers import TimeDistributed, Bidirectional

from keras.optimizers import RMSprop

data_dim = len(letters) + 2
timesteps = 35
num_classes = len(to_phs) + 1
batch_size = 128

model = Sequential()
model.add(LSTM(512, return_sequences=True,
               batch_input_shape=(None, timesteps, data_dim), dropout=0.2, recurrent_dropout=0.1))
model.add(LSTM(512, return_sequences=True, dropout=0.2, recurrent_dropout=0.1, go_backwards=True))
model.add(Bidirectional(LSTM(512, return_sequences=True, dropout=0.2, recurrent_dropout=0.1)))
model.add(LSTM(512, return_sequences=True, dropout=0.2, recurrent_dropout=0.1, go_backwards=True))
model.add(TimeDistributed(Dense(256)))
model.add(Dense(len(to_phs) + 1, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.summary()

x_train = x_data[:85000]
y_train = y_data[:85000]

x_test = x_data[85000:]
y_test = y_data[85000:]


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_14 (LSTM)               (None, 35, 512)           1112064   
_________________________________________________________________
lstm_15 (LSTM)               (None, 35, 512)           2099200   
_________________________________________________________________
bidirectional_4 (Bidirection (None, 35, 1024)          4198400   
_________________________________________________________________
lstm_17 (LSTM)               (None, 35, 512)           3147776   
_________________________________________________________________
time_distributed_2 (TimeDist (None, 35, 256)           131328    
_________________________________________________________________
dense_4 (Dense)              (None, 35, 41)            10537     
Total params: 10,699,305
Trainable params: 10,699,305
Non-trainable params: 0
________________________________________________________________

In [6]:
model.fit(x_train, y_train,
          batch_size=128, epochs=1, shuffle=True)

Epoch 1/1
85000/85000 [==============================] - 6429s - loss: 0.6727 - acc: 0.8313      

In [7]:
model.fit(x_train, y_train,
          batch_size=256, epochs=1, shuffle=True)
model.evaluate(x_test, y_test)
model.fit(x_train, y_train,
          batch_size=512, epochs=1, shuffle=True)
model.evaluate(x_test, y_test)
model.fit(x_train, y_train,
          batch_size=1000, epochs=2, shuffle=True)
model.evaluate(x_test, y_test)

Epoch 1/1
85000/85000 [==============================] - 6235s - loss: 0.3668 - acc: 0.8954     

In [ ]:
model.fit(x_train, y_train,
          batch_size=2000, epochs=1, shuffle=True)

In [ ]:
model.fit(x_train, y_train,
          batch_size=2000, epochs=1, shuffle=True)

In [19]:
model.evaluate(x_test, y_test)

3628/3628 [==============================] - 118s     

[0.13485883771880328, 0.95722948459963819]

In [5]:
y = model.predict(x_test[6].reshape(-1, 35, len(letters) + 2))
print(almost_data[85006])
for i in range(35):
    if (np.max(y[0, i]) >= 1/35):
        print(to_phs[np.argmax(y[0, i])], end='')

NameError: name 'model' is not defined

In [ ]:
x_data = list()
y_data = list()
y_temp = list()
test_file='/Users/AsyaRodigina/hse/SQuAD/SQuAD/lab-5/test.csv'
data_t = np.load(test_file)

x_test = np.empty((len(data_t), 64, 64, 1))
c = 0
for im in data_t:
    im = im / 255.
    im = resize(im, (64, 64), mode='edge')
    x_test[c] = im.reshape((64, 64, 1))
    c +=1
 = model.predict(x_test, batch_size=None)

for i in range(0, len(y_test)):
    result.append([i+1, dic2[np.argmax(y_test[i])]])
import csv
with open('/Users/AsyaRodigina/hse/SQuAD/SQuAD/lab-3/result.csv', 'w') as csvfile:
    writer = csv.writer(csvfile, delimiter=',')
    writer.writerows(result)